### Import Libraries

In [15]:
# Import Libraries
import numpy as np
import pandas as pd

### Dataset Loading

In [16]:
# Load the data
data = pd.read_csv('SBIN_Data.csv')

### Dataset First View

In [17]:
# Dataset First Look
data.head()

,Date,Open,High,Low,Close,Volume,Time
0,01-01-2024,642.20,642.70,641.00,641.95,89481,09:15:00
1,01-01-2024,641.95,642.85,641.95,642.45,48965,09:16:00
2,01-01-2024,642.70,644.00,642.70,643.50,65729,09:17:00
3,01-01-2024,643.90,644.55,643.30,643.40,47909,09:18:00
4,01-01-2024,643.70,644.10,643.45,643.75,27645,09:19:00


In [18]:
data.tail()

,Date,Open,High,Low,Close,Volume,Time
8245,31-01-2024,642.90,642.90,642.50,642.80,161605,15:25:00
8246,31-01-2024,642.90,642.95,642.50,642.90,143774,15:26:00
8247,31-01-2024,642.95,643.05,642.45,642.50,166278,15:27:00
8248,31-01-2024,642.50,642.65,641.40,641.70,138018,15:28:00
8249,31-01-2024,641.70,641.80,641.00,641.55,54342,15:29:00


### Convert Date and Time to a Single DateTime Column

In [19]:
# Specify the format of your datetime strings
datetime_format = '%d-%m-%Y %H:%M:%S'

# Combine 'Date' and 'Time' into a single 'DateTime' column with the correct format
data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format=datetime_format)

# Drop the original 'Date' and 'Time' columns as they are no longer needed
data.drop(['Date', 'Time'], axis=1, inplace=True)

# Set the new 'DateTime' column as the index of your DataFrame
data.set_index('DateTime', inplace=True)



### Calculate the Rank Based on Volume

In [22]:
# Create a new column for the rank
data['Rank'] = 0

# Get unique times and sort them
unique_times = sorted(set(data.index.time))

# Get unique dates and sort them
unique_dates = sorted(set(data.index.date))

# Iterate over each unique time
for time in unique_times:
    # This will hold the last 5 volumes for the current time
    last_5_volumes = []
    for date in unique_dates:
        # Combine date and time to get the full datetime
        datetime_index = pd.Timestamp.combine(date, time)
        # Check if this specific datetime exists in our index (to account for holidays/weekends)
        if datetime_index in data.index:
            # Append the volume to our list
            last_5_volumes.append((datetime_index, data.at[datetime_index, 'Volume']))
            if len(last_5_volumes) > 5:
                # If we have more than 5 volumes, drop the oldest (to maintain the window of 5)
                last_5_volumes.pop(0)
            # Now, rank the volumes within this window of 5
            volumes_only = [x[1] for x in last_5_volumes]
            sorted_volumes = sorted(volumes_only, reverse=True)
            # Find the rank of the most recent volume and update the DataFrame
            if volumes_only:
                latest_volume_rank = sorted_volumes.index(volumes_only[-1]) + 1
                data.at[datetime_index, 'Rank'] = latest_volume_rank

# Optionally, reset the index if you want 'DateTime' back as a column
data.reset_index(inplace=True)

### Save the DataFrame

In [23]:
# Save the DataFrame with the new 'Rank' column
data.to_csv('sbi_stock_data_with_volume_ranks_jan_2024.csv', index=False)

In [24]:
import zipfile

# Specify the path to CSV file
csv_file_path = 'sbi_stock_data_with_volume_ranks_jan_2024.csv'
# Specify the name of the output ZIP file
zip_file_path = 'sbi_stock_data_with_volume_ranks_jan_2024.zip'

# Create a ZIP file and add CSV file to it
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_file_path, arcname='sbi_stock_data_with_volume_ranks_jan_2024.csv')

print(f"File '{csv_file_path}' is compressed to '{zip_file_path}'.")


File 'sbi_stock_data_with_volume_ranks_jan_2024.csv' is compressed to 'sbi_stock_data_with_volume_ranks_jan_2024.zip'.
